In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install gdown --upgrade # Install or upgrade gdown

!gdown --id 1cql0ou71f94Dyqg1bB0K4zVYKJIQX5mQ -O /content/ucf-crime-downsampled.zip

!unzip /content/ucf-crime-downsampled.zip -d /

Streaming output truncated to the last 5000 lines.
 extracting: /content/ucf-crime-downsampled/Test/NormalVideos/Normal_Videos_041_x264_120.png  
 extracting: /content/ucf-crime-downsampled/Test/NormalVideos/Normal_Videos_924_x264_100620.png  
 extracting: /content/ucf-crime-downsampled/Test/NormalVideos/Normal_Videos_935_x264_4130.png  
 extracting: /content/ucf-crime-downsampled/Test/NormalVideos/Normal_Videos_924_x264_71340.png  
 extracting: /content/ucf-crime-downsampled/Test/NormalVideos/Normal_Videos_696_x264_850.png  
  inflating: /content/ucf-crime-downsampled/Test/NormalVideos/Normal_Videos_010_x264_420.png  
 extracting: /content/ucf-crime-downsampled/Test/NormalVideos/Normal_Videos_935_x264_20340.png  
  inflating: /content/ucf-crime-downsampled/Test/NormalVideos/Normal_Videos_938_x264_2200.png  
 extracting: /content/ucf-crime-downsampled/Test/NormalVideos/Normal_Videos_310_x264_1160.png  
 extracting: /content/ucf-crime-downsampled/Test/NormalVideos/Normal_Videos_352_x264

In [3]:
# 2. Imports and Globals
import os, random
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import Sequence, to_categorical
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.layers import (
    Input, Conv2D, DepthwiseConv2D, BatchNormalization,
    ReLU, TimeDistributed, GlobalAveragePooling2D, Dense, Layer
)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


In [12]:
# Paths & Classes
data_root   = "/content/ucf-crime-downsampled"
train_root  = os.path.join(data_root, "Train")
val_root    = os.path.join(data_root, "Test")
class_names = ["Assault","Burglary","Fighting","NormalVideos","Stealing","Vandalism"]

# Hyperparameters
IMG_H, IMG_W = 64, 64
TIMESTEPS    = 16
BATCH_SIZE   = 8
LR           = 3e-4
EPOCHS       = 5

In [13]:
# 3. Data generator: ClipSequence
def list_files(root, cls):
    folder = os.path.join(root, cls)
    return sorted([os.path.join(folder, f) for f in os.listdir(folder)])

class ClipSequence(Sequence):
    def __init__(self, root, classes, batch_size, ts, size):
        self.root    = root
        self.classes = classes
        self.bs      = batch_size
        self.ts      = ts
        self.h, self.w = size
        self.index = {c: list_files(root, c) for c in classes}
        self.pairs = [(c, fp) for c in classes for fp in self.index[c]]
        self.on_epoch_end()

    def __len__(self):
        return len(self.pairs) // self.bs

    def on_epoch_end(self):
        random.shuffle(self.pairs)

    def __getitem__(self, idx):
        batch = self.pairs[idx*self.bs:(idx+1)*self.bs]
        X = np.zeros((self.bs, self.ts, self.h, self.w, 3), np.float32)
        y = np.zeros((self.bs,), np.int32)
        for i, (cls, _) in enumerate(batch):
            y[i] = self.classes.index(cls)
            flist = self.index[cls]
            start = random.randint(0, max(0, len(flist)-self.ts))
            clip_fp = flist[start:start+self.ts]
            if len(clip_fp) < self.ts:
                clip_fp += flist[:self.ts-len(clip_fp)]
            frames = []
            for fp in clip_fp:
                img = tf.keras.preprocessing.image.load_img(fp, target_size=(self.h,self.w))
                arr = tf.keras.preprocessing.image.img_to_array(img)
                frames.append(arr)
            clip = np.stack(frames, axis=0)
            X[i] = preprocess_input(clip)
        return X, to_categorical(y, num_classes=len(self.classes))

In [14]:
# 4. Temporal Shift Module
from tensorflow.keras.layers import Lambda
class TemporalShift(Layer):
    def __init__(self, n_div=8, **kwargs):
        super().__init__(**kwargs)
        self.n_div = n_div

    def call(self, x):
        b = tf.shape(x)[0]
        t = tf.shape(x)[1]
        h = tf.shape(x)[2]
        w = tf.shape(x)[3]
        c = tf.shape(x)[4]
        fold = c // self.n_div
        x1, x2, x3 = tf.split(x, [fold, c - 2 * fold, fold], axis=-1)
        x1 = tf.concat([tf.zeros_like(x1[:, :1]), x1[:, :-1]], axis=1)
        x3 = tf.concat([x3[:, 1:], tf.zeros_like(x3[:, :1])], axis=1)
        return tf.concat([x1, x2, x3], axis=-1)

In [15]:
# 5. Build MobileNetV2+TSM model (fixed backbone instantiation)
from tensorflow.keras.layers import TimeDistributed, Lambda, GlobalAveragePooling2D
import tensorflow.keras.backend as K

def build_mnv2_tsm(ts, h, w, classes):
    inp = Input((ts, h, w, 3), name="clip_input")
    x = TimeDistributed(Conv2D(32, 3, strides=2, padding="same"))(inp)
    x = TimeDistributed(BatchNormalization())(x)
    x = TimeDistributed(ReLU())(x)
    x = TemporalShift(n_div=8)(x)

    # Add a Conv2D layer to reduce the number of channels to 3
    x = TimeDistributed(Conv2D(3, (1, 1), activation='relu'))(x) # Added this line

    base_model = MobileNetV2(include_top=False, weights='imagenet')
    base_model.trainable = False
    x = TimeDistributed(base_model)(x)

    x = TimeDistributed(GlobalAveragePooling2D())(x)
    x = Lambda(lambda y: K.mean(y, axis=1), name="temporal_pooling")(x)
    out = Dense(len(classes), activation="softmax")(x)

    model = Model(inputs=inp, outputs=out)
    model.compile(optimizer=Adam(LR), loss="categorical_crossentropy", metrics=["accuracy"])
    return model

model = build_mnv2_tsm(TIMESTEPS, IMG_H, IMG_W, class_names)
model.summary()

<ipython-input-15-454f66669952>:15: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(include_top=False, weights='imagenet')


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ clip_input (InputLayer)         │ (None, 16, 64, 64, 3)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_18             │ (None, 16, 32, 32, 32) │           896 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_19             │ (None, 16, 32, 32, 32) │           128 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_20             │ (None, 16, 32, 32, 32) │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ temporal_shift_3                │ (None, 16, 32, 32, 32) │             0 │
│ (TemporalShift)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_21             │ (None, 16, 32, 32, 3)  │            99 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_22             │ (None, 16, 1, 1, 1280) │     2,257,984 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_23             │ (None, 16, 1280)       │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ temporal_pooling (Lambda)       │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │         7,686 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,266,793 (8.65 MB)

 Trainable params: 8,745 (34.16 KB)

 Non-trainable params: 2,258,048 (8.61 MB)

In [16]:
# 6. Train with early stopping
train_seq = ClipSequence(train_root, class_names, BATCH_SIZE, TIMESTEPS, (IMG_H,IMG_W))
val_seq   = ClipSequence(val_root,   class_names, BATCH_SIZE, TIMESTEPS, (IMG_H,IMG_W))

callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy', patience= 5, restore_best_weights=True
    )
]
history = model.fit(
    train_seq,
    validation_data=val_seq,
    epochs=EPOCHS,
    callbacks=callbacks
)

Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


3437/3437 ━━━━━━━━━━━━━━━━━━━━ 513s 117ms/step - accuracy: 0.7114 - loss: 1.0128 - val_accuracy: 0.7550 - val_loss: 0.9048
Epoch 2/10
3437/3437 ━━━━━━━━━━━━━━━━━━━━ 365s 106ms/step - accuracy: 0.7437 - loss: 0.7288 - val_accuracy: 0.7255 - val_loss: 0.9251
Epoch 3/10
3437/3437 ━━━━━━━━━━━━━━━━━━━━ 391s 114ms/step - accuracy: 0.7771 - loss: 0.6458 - val_accuracy: 0.6890 - val_loss: 1.0809


In [17]:
model_path = "/content/mnv2_tsm_crime.h5"
model.save(model_path)
print("Saved to", model_path)

Saved to /content/mnv2_tsm_crime.h5


In [19]:
model_path = "/content/mnv2_tsm_crime.keras"
model.save(model_path)
print("Saved to", model_path)

Saved to /content/mnv2_tsm_crime.keras


In [18]:
import shutil

MODEL_PATH = "/content/mnv2_tsm_crime.h5"
DESTINATION_PATH = "/content/drive/MyDrive/NEW/mnv2_tsm_crime.h5"

shutil.copyfile(MODEL_PATH, DESTINATION_PATH)

print(f"Model saved to Google Drive at: {DESTINATION_PATH}")

Model saved to Google Drive at: /content/drive/MyDrive/mnv2_tsm_crime.h5


In [34]:
import shutil

MODEL_PATH = "/content/mnv2_tsm_crime.keras"
DESTINATION_PATH = "/content/drive/MyDrive/NEW/mnv2_tsm_crime.keras"

shutil.copyfile(MODEL_PATH, DESTINATION_PATH)

print(f"Model saved to Google Drive at: {DESTINATION_PATH}")

Model saved to Google Drive at: /content/drive/MyDrive/mnv2_tsm_crime.keras


In [18]:
# 8. Inference snippet
def infer(source=0):
    import cv2
    m = tf.keras.models.load_model('/content/mnv2_tsm_crime.keras', custom_objects={'TemporalShift':TemporalShift})
    cap = cv2.VideoCapture(source)
    buf=[]
    while True:
        ret,frame = cap.read()
        if not ret: break
        buf.append(frame)
        if len(buf)==TIMESTEPS:
            clip = np.stack([cv2.resize(f,(IMG_W,IMG_H)) for f in buf],0)
            clip = preprocess_input(clip)[None]
            p = m.predict(clip)[0]
            idx = p.argmax(); lbl=class_names[idx]
        cv2.putText(frame, lbl, (10,30),1,1,(0,255,0),2)
        cv2.imshow('out',frame); buf=buf[-TIMESTEPS:]
        if cv2.waitKey(1)&0xFF==ord('q'): break
    cap.release(); cv2.destroyAllWindows()

In [31]:
run_inference('/content/Normal_Videos_006_x264.mp4')

ValueError: The `{arg_name}` of this `Lambda` layer is a Python lambda. Deserializing it is unsafe. If you trust the source of the config artifact, you can override this error by passing `safe_mode=False` to `from_config()`, or calling `keras.config.enable_unsafe_deserialization().

In [33]:
# 8. Inference: annotate and display video inline in Colab
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
from tensorflow.keras.models import load_model

model = load_model('/content/mnv2_tsm_crime.keras', custom_objects={'TemporalShift': TemporalShift})

input_path = '/content/Normal_Videos_006_x264.mp4'
out_path = '/content/output_annotated.mp4'

cap = cv2.VideoCapture(input_path)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(out_path, fourcc, fps, (width, height))

buf = []
label, conf = 'Initializing', 0.0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    buf.append(cv2.resize(frame, (IMG_W, IMG_H)))
    if len(buf) == TIMESTEPS:
        clip = np.stack(buf, axis=0)
        clip = preprocess_input(clip)[None]
        p = model.predict(clip, verbose=0)[0]
        idx = np.argmax(p)
        label, conf = class_names[idx], p[idx]
        buf.pop(0)
    text = f"{label}:{conf*100:.1f}%"
    cv2.putText(frame, text, (10, height - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0,255,0), 2)
    out.write(frame)

cap.release()
out.release()

ValueError: The `{arg_name}` of this `Lambda` layer is a Python lambda. Deserializing it is unsafe. If you trust the source of the config artifact, you can override this error by passing `safe_mode=False` to `from_config()`, or calling `keras.config.enable_unsafe_deserialization().